# Decorators

## Introduction

- Decorators is a function that takes anothor function as input and returns a new function that usually extends or alters the behaviour of original one. We can simply say decorator are a way to modify or enhance the behaviour of functions or classes without changing the code.

- Recall the closure example where we have calculated the no of times the function gets called :

  ```python

  def counter(fn, counters):

    count = 0

    def inner(*args,**kwargs):

        nonlocal count

        count += 1

        counters[fn.__name__] = count

        return fn(*args,**kwargs)
    
    return inner

  def add(a,b):

    return a + b

  add = counter(add)

  result = add(1,2)

  ```

  Here we have modified our add function by wrapping it inside the counter function that added the functionality which is counting the no of times add functions gets called. Since counter function changed the functionality of the add function, then we can say `counter` is called as
  Decorator.

- In general a decorator function : 

  1. takes function as argument
  2. returns a closure
  3. the closure usually accepts any combination of parameters
  4. runs some code in the inner function (closure)
  5. the closure function calls the original function using the arguments passed to the closure
  6. returns whatever is returned by the function call.

- So in above example, we say `counter` is a decorator. In general, if `func` is a decorator function, we decorate another function `my_func` using `my_func = func(my_func)`. But python provides another way to decorate a function (not by the assignment we have seen just now.)

  ```python

  @counter
  def add(a,b):

    return a + b
  
  ```

  The above code is same as this

  ```python

  def add(a,b):

    return a + b

  add = counter(add)

  ```

  So we can use `@decorator_function_name` before the function that we actually wanna decorate instead of using that assignment operation we have seen.

In [1]:
# Now lets see the decorators in practice

def add(a,b):

    return a + b

add(1,2)

3

In [3]:
# Now we actually wanna change the functionality of add function, so that it should result the how many times this function gets called.

# For this iam writing a function called counter

def counter(fn):

    count = 0

    def inner(*args,**kwargs):

        nonlocal count

        count += 1

        print(f'Function {fn.__name__} gets called {count} times')
        return fn(*args,**kwargs)
    
    return inner

In [4]:
# Now we call the counter by passing the add function. So it would return a closure which contain the function inner

add = counter(add)

# Now we can try this closure is working correctly or not which means it is performing the add functionality or not.

add(1,2)

Function add gets called 1 times


3

In [7]:
# If you see the retuned closure performing addition operation well.

# As we know instead of using the assignment operation we can use @ for decorating a function

@counter

def add(a,b = 1):

    """
    This add function takes two arguments a and b. This will return addition of a and b.
    """

    return a + b

add(5,6)

Function add gets called 1 times


11

In [8]:
# If you see the above output, the functionality of add gets changed because of the counter decorator.

# But we have a problem here. SInce we have created the counter and add function we know which parameters we need to pass here. 

# But when some other person using this closure, he might not know which parameters need to pass for add function. SO he will use help function to get the function info.

help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [9]:
# But if you see the output , it doesn't contain the signature of the add function. Because from counter function we actually get a closure 
# which contain the function called inner and this add variable references the closure object not actual add function.

# So when we run the help function, it actually returns the doc string or annotations of inner function not add function. This might be a confusion
# when a new user will see this.

# So to replace the signature and annotations of closure (or inner) with the function we actually passing (here passing function is add), we use 
# another decorator called wraps which this passing function as input and decorates the inner function by replacing the singnature, docstring, 
# annotations of inner with the passing function (here it is add).

from functools import wraps

def counter(fn):

    count = 0

    @wraps(fn) # Here it is equivalent to inner = wraps(fn)(inner). We will see these kind of decorators in parameterized decorators.
    def inner(*args,**kwargs):

        nonlocal count

        count += 1

        print(f'Function {fn.__name__} gets called {count} times')

        return fn(*args,**kwargs)
    
    return inner

In [10]:

@counter

def add(a,b = 1):

    """
    This add function takes two arguments a and b. This will return addition of a and b.
    """

    return a + b

add(5,6)

Function add gets called 1 times


11

In [ ]:
# Now lets see the signature of this add function using help function

help(add)

# Here we can see the wraps decorator overwritten the signature of inner with signature of add.

Help on function add in module __main__:

add(a, b=1)
    This add function takes two arguments a and b. This will return addition of a and b.



## Applications

In [1]:
# 1) Timer

# It actually finds the time taken by each function to gets executed

def timed(fn):

    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args,**kwargs):

        start = perf_counter()

        result = fn(*args,**kwargs)

        end = perf_counter()

        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k,v) for (k,v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        print('{0}({1}) took {2:.6f}s to run'.format(fn.__name__,args_str,elapsed))

        return result
    return inner

In [ ]:
# Now lets test the fibbonacci function runtime written in 3 different ways. One is recursive, loop and using reduce function.

@timed
def fibonacci_rescursive(n):

    if n<=2:
        return 1
    return fibonacci_rescursive(n-1) + fibonacci_rescursive(n-2)

In [3]:
fibonacci_rescursive(4)

fibonacci_rescursive(2) took 0.000001s to run
fibonacci_rescursive(1) took 0.000002s to run
fibonacci_rescursive(3) took 0.000708s to run
fibonacci_rescursive(2) took 0.000001s to run
fibonacci_rescursive(4) took 0.000756s to run


3

In [4]:
# Here we have problem, we are  getting the additional runtimes along with actual runtime for finding the fibonacci number 4.
# This is because decorator function gets applied for all the internal recursive functions.

# But we want only time taken for finding the fibbonacci number 4. So to get this, we use another function which calls this fibbonacci function.

def fibonacci_rescursive(n):

    if n<=2:
        return 1
    return fibonacci_rescursive(n-1) + fibonacci_rescursive(n-2)

@timed

def recusive_fibonacci(n):
    return fibonacci_rescursive(n)

In [5]:
recusive_fibonacci(5)

recusive_fibonacci(5) took 0.000005s to run


5

In [7]:
recusive_fibonacci(35)

recusive_fibonacci(35) took 3.075924s to run


9227465

In [8]:
recusive_fibonacci(36)

recusive_fibonacci(36) took 4.553590s to run


14930352

In [9]:
# Now lets check the runtime of fibonacci using the loop

@timed

def fibonacci_loop(n):

    fib_1 = 1
    fib_2 = 1

    for i in range(3,n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    
    return fib_2

In [10]:
fibonacci_loop(4)

fibonacci_loop(4) took 0.000003s to run


3

In [11]:
fibonacci_loop(35)

fibonacci_loop(35) took 0.000007s to run


9227465

In [ ]:
fibonacci_loop(36)

# If you see fibonacci_loop runs faster than recursive function

fibonacci_loop(36) took 0.000008s to run


14930352

In [ ]:
# Now lets do this by using reduce function

from functools import reduce
@timed
def fibonacci_reduce(n):

    fib = reduce(lambda prev,n : (prev[0]+prev[1],prev[0]),range(0,n),(1,0))
    
    return fib[0]

In [16]:
fibonacci_reduce(4)

fibonacci_reduce(4) took 0.000007s to run


5

In [17]:
fibonacci_reduce(35)

fibonacci_reduce(35) took 0.000019s to run


14930352

In [19]:
fibonacci_reduce(36)

fibonacci_reduce(36) took 0.000013s to run


24157817

In [26]:
# Generally if you run same code mutiple times it would give different time becuase of process in cpu. So instead of absolute difference, here we need to consider average elapsed time of certain number iterations.

# So here iam changing the code of timed decorator to incoperate this.

def timed(fn):

    from time import perf_counter
    from functools import wraps


    @wraps(fn)
    def inner(*args,**kwargs):

        elapsed_total = 0
        elapsed_count = 0

        for i in range(10):

            print(f"Running iteration {i} .....")

            start = perf_counter()

            result = fn(*args,**kwargs)

            end = perf_counter()

            elapsed = end - start

            elapsed_total += elapsed

            elapsed_count += 1

        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k,v) for (k,v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        elapsed_avg = elapsed_total/elapsed_count

        print('{0}({1}) took {2:.6f}s to run'.format(fn.__name__,args_str,elapsed_avg))

        return result
    return inner

In [27]:

from functools import reduce
@timed
def fibonacci_reduce(n):

    fib = reduce(lambda prev,n : (prev[0]+prev[1],prev[0]),range(0,n),(1,0))
    
    return fib[0]

In [ ]:
fibonacci_reduce(35)

# Here we are hardcoding the value in for loop. Instead we wanna pass it as value to counter, so that we can find accurate runtime.

# But we cannot use @ notation for creating decorated function with parameters. We need to use parameterized decorators for it.

Running iteration 0 .....
Running iteration 1 .....
Running iteration 2 .....
Running iteration 3 .....
Running iteration 4 .....
Running iteration 5 .....
Running iteration 6 .....
Running iteration 7 .....
Running iteration 8 .....
Running iteration 9 .....
fibonacci_reduce(35) took 0.000009s to run


14930352